In [1]:
!pip install flask flask-cors pandas numpy scikit-learn xgboost lightgbm joblib pyngrok


In [2]:
!pip install pyngrok

In [3]:
!mkdir -p models data

In [4]:
import os
import sys
import json
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from google.colab import drive, files

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
json_files = ['states.json', 'districts.json', 'markets.json',
              'commodities.json', 'varieties.json', 'grades.json']

In [ ]:
for file in json_files:
    try:
        drive_path = f'/content/drive/MyDrive/data/{file}' 
        if os.path.exists(drive_path):
            print(f"Copying {file} from Drive...")
            !cp "{drive_path}" "data/{file}"
        else:
            print(f"WARNING: {file} not found in Drive. Please upload it.")
            print(f"Please upload {file}...")
            uploaded = files.upload()
            if file in uploaded:
                !mv "{file}" "data/{file}"
    except Exception as e:
        print(f"Error copying {file}: {str(e)}")

Copying states.json from Drive...
Copying districts.json from Drive...
Copying markets.json from Drive...
Copying commodities.json from Drive...
Copying varieties.json from Drive...
Copying grades.json from Drive...


In [9]:
model_files = [
    'min_price_xgb_model_10.joblib',
    'max_price_xgb_model_10.joblib',
    'stage2_avg_price_lgb_model_10.joblib',
    'scaler.pkl'
]

In [ ]:
for file in model_files:
    try:
        drive_path = f'/content/drive/MyDrive/models/{file}' 
        if os.path.exists(drive_path):
            print(f"Copying {file} from Drive...")
            !cp "{drive_path}" "models/{file}"
        else:
            print(f"WARNING: {file} not found in Drive. Please upload it.")
            print(f"Please upload {file}...")
            uploaded = files.upload()
            if file in uploaded:
                !mv "{file}" "models/{file}"
    except Exception as e:
        print(f"Error copying {file}: {str(e)}")

Copying min_price_xgb_model_10.joblib from Drive...
Copying max_price_xgb_model_10.joblib from Drive...
Copying stage2_avg_price_lgb_model_10.joblib from Drive...
Copying scaler.pkl from Drive...


In [ ]:

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

models = {
    'min_price': None,
    'max_price': None,
    'modal_price': None,
    'scaler': None,
    'encoders': None
}


def create_label_encoders():
    encoders = {}

    with open('data/states.json', 'r') as f:
        states_list = json.load(f)

    with open('data/districts.json', 'r') as f:
        districts_map = json.load(f)

    with open('data/markets.json', 'r') as f:
        markets_map = json.load(f)

    with open('data/commodities.json', 'r') as f:
        commodities_list = json.load(f)

    with open('data/varieties.json', 'r') as f:
        varieties_map = json.load(f)

    with open('data/grades.json', 'r') as f:
        grades_map = json.load(f)

    # State encoder
    state_encoder = LabelEncoder()
    state_encoder.fit(states_list)
    encoders['State'] = state_encoder

    # District encoder
    all_districts = []
    for district_list in districts_map.values():
        all_districts.extend(district_list)
    district_encoder = LabelEncoder()
    district_encoder.fit(list(set(all_districts)))
    encoders['District'] = district_encoder

    # Market encoder
    all_markets = []
    for market_list in markets_map.values():
        all_markets.extend(market_list)
    market_encoder = LabelEncoder()
    market_encoder.fit(list(set(all_markets)))
    encoders['Market'] = market_encoder

    # Commodity encoder
    commodity_encoder = LabelEncoder()
    commodity_encoder.fit(commodities_list)
    encoders['Commodity'] = commodity_encoder

    # Variety encoder
    all_varieties = []
    for variety_list in varieties_map.values():
        all_varieties.extend(variety_list)
    variety_encoder = LabelEncoder()
    variety_encoder.fit(list(set(all_varieties)))
    encoders['Variety'] = variety_encoder

    # Grade encoder
    all_grades = []
    for grade_list in grades_map.values():
        all_grades.extend(grade_list)
    grade_encoder = LabelEncoder()
    grade_encoder.fit(list(set(all_grades)))
    encoders['Grade'] = grade_encoder

    return encoders

def load_models():
    """Load all machine learning models and preprocessors"""
    print("Loading models...")
    try:
        # Load encoders
        models['encoders'] = create_label_encoders()

        # Load the scaler
        with open('models/scaler.pkl', 'rb') as f:
            models['scaler'] = pickle.load(f)

        # Load the prediction models
        models['min_price'] = joblib.load('models/min_price_xgb_model_10.joblib')
        models['max_price'] = joblib.load('models/max_price_xgb_model_10.joblib')
        models['modal_price'] = joblib.load('models/stage2_avg_price_lgb_model_10.joblib')

        print("Models loaded successfully")
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        raise

@app.route('/get_states', methods=['GET'])
def get_states():
    with open('data/states.json', 'r') as f:
        states_list = json.load(f)
    return jsonify({"states": states_list})

@app.route('/get_districts', methods=['GET'])
def get_districts():
    state = request.args.get('state')
    with open('data/districts.json', 'r') as f:
        districts_map = json.load(f)
    if state in districts_map:
        return jsonify({"districts": districts_map[state]})
    return jsonify({"districts": []})

@app.route('/get_markets', methods=['GET'])
def get_markets():
    district = request.args.get('district')
    with open('data/markets.json', 'r') as f:
        markets_map = json.load(f)
    if district in markets_map:
        return jsonify({"markets": markets_map[district]})
    return jsonify({"markets": []})

@app.route('/get_commodities', methods=['GET'])
def get_commodities():
    with open('data/commodities.json', 'r') as f:
        commodities_list = json.load(f)
    return jsonify({"commodities": commodities_list})

@app.route('/get_varieties', methods=['GET'])
def get_varieties():
    commodity = request.args.get('commodity')
    with open('data/varieties.json', 'r') as f:
        varieties_map = json.load(f)
    if commodity in varieties_map:
        return jsonify({"varieties": varieties_map[commodity]})
    return jsonify({"varieties": []})

@app.route('/get_grades', methods=['GET'])
def get_grades():
    variety = request.args.get('variety')
    with open('data/grades.json', 'r') as f:
        grades_map = json.load(f)
    if variety in grades_map:
        return jsonify({"grades": grades_map[variety]})
    return jsonify({"grades": []})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # If models are not loaded, load them
        if models['min_price'] is None:
            load_models()

        # Get input data from request
        input_data = request.json
        print(f"Received input data: {input_data}")

        # Create a DataFrame with the input features
        input_df = pd.DataFrame([{
            'State': input_data['State'],
            'District': input_data['District'],
            'Market': input_data['Market'],
            'Commodity': input_data['Commodity'],
            'Variety': input_data['Variety'],
            'Grade': input_data['Grade'],
            'Year': input_data['Year'],
            'Month': input_data['Month'],
            'Day': input_data['Day']
        }])

        # Encode categorical features
        for col in ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade']:
            encoder = models['encoders'][col]

            # Handle unseen values (important for production)
            try:
                input_df[col] = encoder.transform(input_df[col])
            except ValueError:
                # If value not in encoder, use most common value
                print(f"Warning: Unknown value in {col}: {input_df[col].values[0]}")
                input_df[col] = encoder.transform([encoder.classes_[0]])

        # Scale numerical features
        input_array = models['scaler'].transform(input_df)

        # Stage 1: Predict min and max prices
        min_price = models['min_price'].predict(input_array)[0]
        max_price = models['max_price'].predict(input_array)[0]

        # Stage 2: Add min and max price predictions to input features
        input_df['Min_Price'] = min_price
        input_df['Max_Price'] = max_price

        # Predict modal price
        modal_price = models['modal_price'].predict(input_df)[0]

        # Return predictions
        response = {
            'min_price': float(min_price),
            'max_price': float(max_price),
            'modal_price': float(modal_price)
        }

        print(f"Prediction response: {response}")
        return jsonify(response)

    except Exception as e:
        error_message = f"Error in prediction: {str(e)}"
        print(error_message)
        return jsonify({"error": error_message}), 500

#  Set up ngrok
ngrok_auth_token = '2vXsqr5ns4G6ciW2UKseTFWhE2u_5ytH5jjATvMXnfWq4wVvX'
ngrok.set_auth_token(ngrok_auth_token)

# Verify model files
for file in model_files:
    if not os.path.exists(f"models/{file}"):
        print(f"WARNING: {file} not found in models directory!")

# Verify data files 
for file in json_files:
    if not os.path.exists(f"data/{file}"):
        print(f"WARNING: {file} not found in data directory!")

# Load models at startup
try:
    load_models()
    print("Models loaded successfully at startup")
except Exception as e:
    print(f"Error loading models at startup: {str(e)}")

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Loading models...
Models loaded successfully
Models loaded successfully at startup
Public URL: NgrokTunnel: "https://ed76-34-106-128-102.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:29] "GET /health HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:29] "GET /get_states HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:29] "GET /get_commodities HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:42] "GET /get_districts?state=Haryana HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:44] "GET /get_markets?district=Karnal HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:48] "GET /get_varieties?commodity=Anthorium HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:50] "GET /get_grades?variety=Anthorium HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 05:59:59] "POST

Received input data: {'State': 'Haryana', 'District': 'Karnal', 'Market': 'Kunjpura', 'Commodity': 'Anthorium', 'Variety': 'Anthorium', 'Grade': 'FAQ', 'Year': 2025, 'Month': 4, 'Day': 17}
Prediction response: {'min_price': 1535.746826171875, 'max_price': 5328.0966796875, 'modal_price': 4407.075872824059}


INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:11:15] "GET /get_districts?state=Gujarat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:11:17] "GET /get_markets?district=Devbhumi%20Dwarka HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:11:20] "GET /get_varieties?commodity=Apple HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:11:22] "GET /get_grades?variety=Kesri HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:11:25] "POST /predict HTTP/1.1" 200 -


Received input data: {'State': 'Gujarat', 'District': 'Devbhumi Dwarka', 'Market': 'Bhanvad', 'Commodity': 'Apple', 'Variety': 'Kesri', 'Grade': 'Large', 'Year': 2025, 'Month': 4, 'Day': 17}
Prediction response: {'min_price': 1535.746826171875, 'max_price': 5328.0966796875, 'modal_price': 4091.021471596665}


...

Received input data: {'State': 'Tamil Nadu', 'District': 'Madurai', 'Market': 'Madurai', 'Commodity': 'Apple', 'Variety': 'Kesri', 'Grade': 'Large', 'Year': 2025, 'Month': 4, 'Day': 17}
Prediction response: {'min_price': 1167.9197998046875, 'max_price': 3128.768798828125, 'modal_price': 2260.689826772746}


INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:44:20] "POST /predict HTTP/1.1" 200 -


Received input data: {'State': 'Tamil Nadu', 'District': 'Madurai', 'Market': 'Madurai', 'Commodity': 'Apple', 'Variety': 'Kesri', 'Grade': 'Medium', 'Year': 2025, 'Month': 4, 'Day': 17}
Prediction response: {'min_price': 1497.8123779296875, 'max_price': 1072.3587646484375, 'modal_price': 2091.505095273905}


INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 06:45:33] "POST /predict HTTP/1.1" 200 -


Received input data: {'State': 'Tamil Nadu', 'District': 'Madurai', 'Market': 'Madurai', 'Commodity': 'Apple', 'Variety': 'Kesri', 'Grade': 'Medium', 'Year': 2025, 'Month': 4, 'Day': 17}
Prediction response: {'min_price': 1497.8123779296875, 'max_price': 1072.3587646484375, 'modal_price': 2091.505095273905}
